In [19]:
from google.colab import drive
drive.mount("/content/drive")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('my_app').getOrCreate()
bank_copy=spark.read.csv('/content/ML_hw_dataset (1).csv',header=True,inferSchema=True)

In [5]:
#bank_copy.select("age","job").show()
#bank_copy.describe().show()
#bank_copy.dtypes


bank_copy.show()


+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|        education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 44|blue-collar| married|         basic.4y|unknown|    yes|  no| cellular|  aug|        thu|     210|       1|  999|       0|nonexistent|         1.4|        93.444|        -36.1|    4.963|     5228.1|  0|
| 53| technician| married|          unknown|     no|     no|  no| cellular|  nov|        fri|     138|       1|  999|       0|nonexistent|        -0.1|          93.2|      


missing value

In [6]:
#Null & NAN
from pyspark.sql.functions import isnan,when,count,col

# ساخت یک دیکشنری خالی برای نگه داری تعداد مقادیر نا معتبر هر ستون
null_counts = {}

# بررسی تمام ستون‌های دیتافریم
for i in bank_copy.columns:
    # شمارش تعداد مقادیر نا معتبر در هر ستون و ذخیره آن در دیکشنری
    null_counts[i] = bank_copy.filter(col(i).isNull() | isnan(i)).count()

# چاپ دیکشنری حاوی تعداد مقادیر نا معتبر هر ستون
print(null_counts)

{'age': 0, 'job': 0, 'marital': 0, 'education': 0, 'default': 0, 'housing': 0, 'loan': 0, 'contact': 0, 'month': 0, 'day_of_week': 0, 'duration': 0, 'campaign': 0, 'pdays': 0, 'previous': 0, 'poutcome': 0, 'emp_var_rate': 0, 'cons_price_idx': 0, 'cons_conf_idx': 0, 'euribor3m': 0, 'nr_employed': 0, 'y': 0}


ما 4118 نمونه و 21 ویژگی داریم. اطلاعات می گوید هیچ مقدار تهی وجود ندارد. 


************************************************************************

outlier    ==> روش IQR

In [7]:
"""from pyspark.sql.functions import col

numeric_cols = [c for c, t in bank_copy.dtypes if t in ['int', 'double', 'float', 'long']]

# محاسبه‌ی محدوده‌ی IQR
for col_name in numeric_cols:
    quantiles = bank_copy.approxQuantile(col_name, [0.25, 0.75], 0.05)
    q1 = quantiles[0]
    q3 = quantiles[1]
    iqr = q3 - q1

# محاسبه‌ی مقدار حد پایین و حد بالای محدوده‌ی IQR
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# تشخیص داده‌هایی که خارج از محدوده‌ی IQR هستند
#outliers = bank_copy.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound))
#outliers.show()


bank_copy = bank_copy.filter((col(col_name) >= lower_bound) & (col(col_name) <= upper_bound))
bank_copy.show()"""

"from pyspark.sql.functions import col\n\nnumeric_cols = [c for c, t in bank_copy.dtypes if t in ['int', 'double', 'float', 'long']]\n\n# محاسبه\u200cی محدوده\u200cی IQR\nfor col_name in numeric_cols:\n    quantiles = bank_copy.approxQuantile(col_name, [0.25, 0.75], 0.05)\n    q1 = quantiles[0]\n    q3 = quantiles[1]\n    iqr = q3 - q1\n\n# محاسبه\u200cی مقدار حد پایین و حد بالای محدوده\u200cی IQR\nlower_bound = q1 - 1.5 * iqr\nupper_bound = q3 + 1.5 * iqr\n\n# تشخیص داده\u200cهایی که خارج از محدوده\u200cی IQR هستند\n#outliers = bank_copy.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound))\n#outliers.show()\n\n\nbank_copy = bank_copy.filter((col(col_name) >= lower_bound) & (col(col_name) <= upper_bound))\nbank_copy.show()"

In [8]:
bank_copy.show()

+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|        education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 44|blue-collar| married|         basic.4y|unknown|    yes|  no| cellular|  aug|        thu|     210|       1|  999|       0|nonexistent|         1.4|        93.444|        -36.1|    4.963|     5228.1|  0|
| 53| technician| married|          unknown|     no|     no|  no| cellular|  nov|        fri|     138|       1|  999|       0|nonexistent|        -0.1|          93.2|      

************************************************************************


StringIndexer

In [9]:
from pyspark.ml.feature import StringIndexer

# تعریف لیستی از نام ستون‌های مورد نظر
columns_to_index = ["job", "marital","education",'default','housing','loan','contact','month','day_of_week','poutcome']
# ایجاد یک شی از کلاس StringIndexer برای هر ستون و اعمال آن بر روی داده
for column in columns_to_index:
    indexer = StringIndexer(inputCol=column, outputCol=column+"_N")
    bank_copy = indexer.fit(bank_copy).transform(bank_copy)

# نمایش دیتافریم جدید
bank_copy.show()

+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+-----+---------+-----------+---------+---------+------+---------+-------+-------------+----------+
|age|        job| marital|        education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|job_N|marital_N|education_N|default_N|housing_N|loan_N|contact_N|month_N|day_of_week_N|poutcome_N|
+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+-----+---------+-----------+---------+---------+------+---------+-------+-------------+----------+
| 44|blue-collar| married|         basic.4y|unknown|    yes|  no| cellular|  aug|    

correlation

In [10]:
from pyspark.sql.functions import corr
numeric_cols=["age","duration","campaign","pdays","previous","emp_var_rate","cons_price_idx","cons_conf_idx","euribor3m","nr_employed",'job_N','marital_N','education_N','default_N','housing_N','loan_N','contact_N','month_N','day_of_week_N','poutcome_N','y']
l=[]
for col in numeric_cols:
    correlation = bank_copy.select(corr("y", col)).collect()[0][0]
    if correlation is not None and (correlation>0.1 or correlation<-0.1):
        l.append(col)
        print(f"The correlation between 'label' and {col} is {correlation}")
print(l)



The correlation between 'label' and duration is 0.4052738006793088
The correlation between 'label' and pdays is -0.32491447761663295
The correlation between 'label' and previous is 0.23018100321659
The correlation between 'label' and emp_var_rate is -0.2983344261593787
The correlation between 'label' and cons_price_idx is -0.13621121281925005
The correlation between 'label' and euribor3m is -0.3077714039407199
The correlation between 'label' and nr_employed is -0.35467829592143435
The correlation between 'label' and contact_N is -0.14477305571200028
The correlation between 'label' and month_N is 0.22268368982305878
The correlation between 'label' and poutcome_N is 0.2703731053271902
The correlation between 'label' and y is 1.0
['duration', 'pdays', 'previous', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed', 'contact_N', 'month_N', 'poutcome_N', 'y']


In [11]:
R=['duration', 'pdays', 'previous', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed', 'contact_N', 'month_N', 'poutcome_N']

In [12]:
bank_copy.select(R).show()


+--------+-----+--------+------------+--------------+---------+-----------+---------+-------+----------+
|duration|pdays|previous|emp_var_rate|cons_price_idx|euribor3m|nr_employed|contact_N|month_N|poutcome_N|
+--------+-----+--------+------------+--------------+---------+-----------+---------+-------+----------+
|     210|  999|       0|         1.4|        93.444|    4.963|     5228.1|      0.0|    2.0|       0.0|
|     138|  999|       0|        -0.1|          93.2|    4.021|     5195.8|      0.0|    4.0|       0.0|
|     339|    6|       2|        -1.7|        94.055|    0.729|     4991.6|      0.0|    3.0|       2.0|
|     185|  999|       0|        -1.8|        93.075|    1.405|     5099.1|      0.0|    5.0|       0.0|
|     137|    3|       1|        -2.9|        92.201|    0.869|     5076.2|      0.0|    2.0|       2.0|
|      68|  999|       0|         1.4|        93.918|    4.961|     5228.1|      0.0|    1.0|       0.0|
|     204|  999|       0|        -1.8|        92.893|  

VectorAssembler

In [13]:
from pyspark.ml.feature import VectorAssembler

# ستون‌هایی که باید وارد VectorAssembler شوند
inputCols = R

# نام ستونی که در خروجی VectorAssembler قرار میگیرد
outputCol = "assembled_features_new"

# ساخت یک شی از کلاس VectorAssembler
assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)

# تبدیل داده‌های ورودی به فرمت VectorAssembler
bank_copy = assembler.transform(bank_copy)

# نمایش دیتافریم جدید
#bank_copy.select(features).show()
bank_copy.show()

+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+-----+---------+-----------+---------+---------+------+---------+-------+-------------+----------+----------------------+
|age|        job| marital|        education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|job_N|marital_N|education_N|default_N|housing_N|loan_N|contact_N|month_N|day_of_week_N|poutcome_N|assembled_features_new|
+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+-----+---------+-----------+---------+---------+------+---------+-------+-------------+----------+----------------------+
| 44|blue-collar

StandardScaler

In [14]:
from pyspark.ml.feature import StandardScaler

# ستونی که باید استانداردسازی شود
inputCol = "assembled_features_new"

# نام ستونی که در خروجی StandardScaler قرار می‌گیرد
outputCol = "standard_features"

# ساخت یک شی از کلاس StandardScaler
scaler = StandardScaler(inputCol=inputCol, outputCol=outputCol)

# تبدیل داده‌های ورودی به فرمت استاندارد شده
scalerModel = scaler.fit(bank_copy)
bank_copy = scalerModel.transform(bank_copy)

# نمایش دیتافریم جدید
bank_copy.show()

+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+-----+---------+-----------+---------+---------+------+---------+-------+-------------+----------+----------------------+--------------------+
|age|        job| marital|        education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|job_N|marital_N|education_N|default_N|housing_N|loan_N|contact_N|month_N|day_of_week_N|poutcome_N|assembled_features_new|   standard_features|
+---+-----------+--------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+-----+---------+-----------+---------+---------+------+---------+-------+-------------+---------

LogisticRegression

In [16]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

train_data, test_data =bank_copy.randomSplit([0.8, 0.2], seed=100)
accuracy=0
Recall=0
Precision=0
F_Measure=0
n=10
for i in range(n):
  lr = LogisticRegression(labelCol="y", featuresCol="standard_features")
  lrModel = lr.fit(train_data)
  predictions = lrModel.transform(test_data)
  #predictions.show()
  evaluator = BinaryClassificationEvaluator(labelCol="y",rawPredictionCol="rawPrediction")
  predictions = lrModel.transform(test_data)
  predictions.groupBy('y', 'prediction').count().show()
  TN = predictions.filter('prediction = 0 AND y = prediction').count()
  TP = predictions.filter('prediction = 1 AND y = prediction').count()
  FN = predictions.filter('prediction = 0 AND y = 1').count()
  FP = predictions.filter('prediction = 1 AND y = 0').count()
  accuracy =accuracy+ (TN + TP) / (TN + TP + FN + FP)
  Precision =Precision+ TP/(TP+FP)
  Recall =Recall+ TP/(TP+FN)
  F_Measure=F_Measure+(2*Precision*Recall)/(Precision+Recall)


print("Accuracy:", accuracy / n)
print("Precision:", Precision/n)
print("Recall:", Recall / n)
print("F-measure:", F_Measure / n)

+---+----------+-----+
|  y|prediction|count|
+---+----------+-----+
|  1|       0.0|  519|
|  0|       0.0| 7128|
|  1|       1.0|  373|
|  0|       1.0|  200|
+---+----------+-----+

+---+----------+-----+
|  y|prediction|count|
+---+----------+-----+
|  1|       0.0|  519|
|  0|       0.0| 7128|
|  1|       1.0|  373|
|  0|       1.0|  200|
+---+----------+-----+

+---+----------+-----+
|  y|prediction|count|
+---+----------+-----+
|  1|       0.0|  519|
|  0|       0.0| 7128|
|  1|       1.0|  373|
|  0|       1.0|  200|
+---+----------+-----+

+---+----------+-----+
|  y|prediction|count|
+---+----------+-----+
|  1|       0.0|  519|
|  0|       0.0| 7128|
|  1|       1.0|  373|
|  0|       1.0|  200|
+---+----------+-----+

+---+----------+-----+
|  y|prediction|count|
+---+----------+-----+
|  1|       0.0|  519|
|  0|       0.0| 7128|
|  1|       1.0|  373|
|  0|       1.0|  200|
+---+----------+-----+

+---+----------+-----+
|  y|prediction|count|
+---+----------+-----+
|  1| 

LinearSVC

در اینجا، ۱۰ بار اجرای مدل را در نظر گرفته‌ایم

In [18]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

train_data, test_data = bank_copy.randomSplit([0.8, 0.2], seed=10)

accuracy = 0
recall = 0
precision = 0
f_measure = 0
n = 10

for i in range(n):
    # ایجاد یک شی از کلاس LinearSVC
    lsvc = LinearSVC(labelCol="y", featuresCol="standard_features", maxIter=10)

    # آموزش مدل با داده‌های آموزشی
    lsvcModel = lsvc.fit(train_data)

    # اعمال مدل بر داده‌های تست
    predictions = lsvcModel.transform(test_data)

    # محاسبه معیارات ارزیابی
    evaluator = BinaryClassificationEvaluator(labelCol="y", rawPredictionCol="rawPrediction")
    tn = predictions.filter('prediction = 0 AND y = prediction').count()
    tp = predictions.filter('prediction = 1 AND y = prediction').count()
    fn = predictions.filter('prediction = 0 AND y = 1').count()
    fp = predictions.filter('prediction = 1 AND y = 0').count()
    accuracy += (tn + tp) / (tn + tp + fn + fp)
    precision += tp / (tp + fp)
    recall += tp / (tp + fn)
    f_measure += (2 * precision * recall) / (precision + recall)

print("Accuracy:", accuracy / n)
print("Precision:", precision / n)
print("Recall:", recall / n)
print("F-measure:", f_measure / n)


Accuracy: 0.9037019114444715
Precision: 0.6059322033898306
Recall: 0.3191964285714285
F-measure: 2.299707602339181
